# Visualize using FiftyOne

# Load Dataset

In [2]:
!pip install -q fiftyone

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 7.8 MB 34.3 MB/s eta 0:00:01
     |████████████████████████████████| 570 kB 168.7 MB/s eta 0:00:01
     |████████████████████████████████| 192 kB 173.3 MB/s eta 0:00:01
     |████████████████████████████████| 108 kB 173.7 MB/s eta 0:00:01
     |████████████████████████████████| 79.9 MB 172.7 MB/s eta 0:00:01
     |████████████████████████████████| 41 kB 46.2 MB/s  eta 0:00:01
     |████████████████████████████████| 78 kB 146.2 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 167.2 MB/s eta 0:00:01
     |████████████████████████████████| 37.8 MB 161.3 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 129.9 MB/s eta 0:00:01
     |████████████████████████████████| 15.6 MB 167.6 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 147.1 MB/s eta 0:00:01
     |████████████████████████████████| 53 kB 94.7 MB/s  eta 0:00:01
     |█████████████

In [3]:
import os
import fiftyone as fo
from tqdm.notebook import tqdm
import json
from hoarder.datasets.local_dataset import LocalDataset

dataset_dir = "sample_data/TestSAM/"

ld = LocalDataset(dataset_dir)

Migrating database to v0.22.3


In [19]:
import numpy as np
image_frames = ld.get_frames()
samples = []

for img_idx, frame in tqdm(enumerate(image_frames), total=len(image_frames)):
    image_path = ld.get_local_image_path(frame)
    annotations = frame['annotations']

    if not os.path.exists(image_path):
        print(image_path)
        continue

    metadata = fo.ImageMetadata.build_for(image_path)

    detections = []
    polylines = []
    for idx, ann in enumerate(annotations):
        label = ann["labels"][0]
        if "boundingBox" in ann:
            # ld.pull()
            boxxywh = ann["boundingBox"]
            boxxywh = np.array([boxxywh['x'], boxxywh['y'], boxxywh['w'], boxxywh['h']]).astype(np.float32)
            boxxywh[[0,2]] /= metadata['width']
            boxxywh[[1,3]] /= metadata['height']

            detections.append(
                fo.Detection(
                    label=label,
                    bounding_box=boxxywh.tolist(),
                )
            )

        if "boundingPolygon" in ann:
            polygon = ann["boundingPolygon"]
            polygon = [[data['x'], data['y']] for data in polygon]
            polygon = np.array(polygon).reshape((-1,2)).astype(np.float32)
            polygon /= np.array([metadata['width'], metadata['height']]).reshape((1,2))
            polygon = [tuple(pair.tolist()) for pair in polygon]

            polylines.append(fo.Polyline(label=label, points=[polygon], closed=False, filled=False))

    sample = fo.Sample(
        filepath=image_path,
        metadata=metadata
    )
    
    # Update to sample fields
    # detections as detections
    # polylines as polylines
    sample["detections"] = fo.Detections(detections=detections)
    sample["polylines"] = fo.Polylines(polylines=polylines)

    samples.append(sample)

  0%|          | 0/300 [00:00<?, ?it/s]

In [20]:
# Load from dataset
# dataset = fo.load_dataset("fiftyone_testSAM")

# Create new dataset
# dataset = fo.Dataset("fiftyone_testSAM")

# Clear dataset 
dataset.clear()
dataset.add_samples(samples)


 100% |█████████████████| 300/300 [16.8s elapsed, 0s remaining, 16.5 samples/s]      


['65588d174f7a974a058bea98',
 '65588d174f7a974a058bea99',
 '65588d174f7a974a058bea9a',
 '65588d174f7a974a058bea9b',
 '65588d174f7a974a058bea9c',
 '65588d174f7a974a058bea9d',
 '65588d174f7a974a058bea9e',
 '65588d174f7a974a058bea9f',
 '65588d174f7a974a058beaa0',
 '65588d174f7a974a058beaa1',
 '65588d174f7a974a058beaa2',
 '65588d184f7a974a058beaa3',
 '65588d184f7a974a058beaa4',
 '65588d184f7a974a058beaa5',
 '65588d184f7a974a058beaa6',
 '65588d184f7a974a058beaa7',
 '65588d184f7a974a058beaa8',
 '65588d184f7a974a058beaa9',
 '65588d184f7a974a058beaaa',
 '65588d184f7a974a058beaab',
 '65588d194f7a974a058beaac',
 '65588d194f7a974a058beaad',
 '65588d194f7a974a058beaae',
 '65588d194f7a974a058beaaf',
 '65588d194f7a974a058beab0',
 '65588d194f7a974a058beab1',
 '65588d194f7a974a058beab2',
 '65588d194f7a974a058beab3',
 '65588d194f7a974a058beab4',
 '65588d194f7a974a058beab5',
 '65588d194f7a974a058beab6',
 '65588d194f7a974a058beab7',
 '65588d194f7a974a058beab8',
 '65588d194f7a974a058beab9',
 '65588d194f7a

In [21]:
dataset.export(
    dataset_type=fo.types.FiftyOneDataset,
    export_dir="sample_data/fiftyone_testSAM",
)

Directory 'sample_data/fiftyone_testSAM' already exists; export will be merged with existing files
Exporting samples...
 100% |████████████████████| 300/300 [3.3s elapsed, 0s remaining, 93.6 docs/s]       


In [17]:
session = fo.launch_app(dataset.limit(150))



Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.22.3

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



In [18]:
session.show()